In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [11]:

transforms_func = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
train_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets/train',
    transform=transforms_func
)
val_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets/val',
    transform=transforms_func
)



data_dir = 'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets'
batch_size = 32
train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=False)


In [12]:
import matplotlib.pyplot as plt
import numpy as np

# Get a single image and label from the training dataset
img, lbl =train_dataloader[0] # Index can be any valid integer

# Convert the image from (C, H, W) to (H, W, C)
img = img.permute(1, 2, 0)



plt.imshow(img)
plt.title(f'Label: {lbl}')

plt.show()

TypeError: 'DataLoader' object is not subscriptable

In [ ]:


class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.features_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten()
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 13 * 13, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 25)
        )

    def forward(self, x):
        x = self.features_extractor(x)
        x = self.classifier(x)
        return x



In [ ]:
import torch
from tqdm import tqdm

def train(model, 
          criteria,  # Use CrossEntropyLoss for multi-class classification
          optimizer, 
          train_dataloader, 
          val_dataloader, 
          epochs, 
          device, 
          val_iteration=1):
    train_loss = []
    val_loss = []
    train_accuracy = []
    val_accuracy = []

    for epoch in range(1, epochs + 1):
        ## Train 
        model.train()
        epoch_train_loss = 0
        epoch_train_correct = 0
        train_step = 0
        for batch_images, batch_labels in tqdm(train_dataloader):
            batch_images = batch_images.to(device ,non_blocking=True)
            batch_labels = batch_labels.to(device ,non_blocking=True)

            optimizer.zero_grad()
            # forward pass
            results = model(batch_images)
            loss = criteria(results, batch_labels)  # CrossEntropyLoss expects raw logits

            # backward pass
            loss.backward()  # calculate gradients
            optimizer.step()  # update weights using gradients

            train_step += 1
            epoch_train_loss += loss.item()

            # Training accuracy calculation
            _, predicted = torch.max(results, 1)  # Get the index of the max log-probability
            epoch_train_correct += (predicted == batch_labels).sum().item()

        epoch_train_loss /= train_step
        epoch_train_accuracy = epoch_train_correct / len(train_dataloader.dataset)
        train_loss.append(epoch_train_loss)
        train_accuracy.append(epoch_train_accuracy)
        print(f"Epoch {epoch} of {epochs}: train loss = {epoch_train_loss:.4f}, train accuracy = {epoch_train_accuracy:.4f}")

        ## Validation 
        if epoch % val_iteration == 0:
            model.eval()

            epoch_val_loss = 0
            epoch_val_correct = 0
            val_step = 0
            with torch.no_grad():
                for batch_images, batch_labels in val_dataloader:
                    batch_images = batch_images.to(device ,non_blocking=True)
                    batch_labels = batch_labels.to(device ,non_blocking=True)
                    
                    # forward pass
                    results = model(batch_images)
                    loss = criteria(results, batch_labels)
                    epoch_val_loss += loss.item()
                    val_step += 1

                    # Validation accuracy calculation
                    _, predicted = torch.max(results, 1)  # Get the index of the max log-probability
                    epoch_val_correct += (predicted == batch_labels).sum().item()

            epoch_val_loss /= val_step
            epoch_val_accuracy = epoch_val_correct / len(val_dataloader.dataset)
            val_loss.append(epoch_val_loss)
            val_accuracy.append(epoch_val_accuracy)
            print(f"Epoch {epoch} of {epochs}: validation loss = {epoch_val_loss:.4f}, validation accuracy = {epoch_val_accuracy:.4f}")

    return train_loss, val_loss, train_accuracy, val_accuracy


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [ ]:
epochs = 10 

model = CustomCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Define loss function and optimizer


In [13]:
train_loss, val_loss, train_accuracy, val_accuracy = train(
    model,
    criteria=criterion,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=epochs,
    device=device
)

  0%|          | 0/704 [00:00<?, ?it/s]

100%|██████████| 704/704 [53:32<00:00,  4.56s/it]  


Epoch 1 of 10: train loss = 2.5663, train accuracy = 0.1556
Epoch 1 of 10: validation loss = 2.2890, validation accuracy = 0.2289


  2%|▏         | 17/704 [01:24<56:46,  4.96s/it]


KeyboardInterrupt: 

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os

# Transformations
transforms_func = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets
train_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets/train',
    transform=transforms_func
)
val_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets/val',
    transform=transforms_func
)

# Dataloaders with increased num_workers
data_dir = 'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets'
batch_size = 32 # Increased batch size for faster training
train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=False, num_workers=4)

# Define a custom CNN model
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.features_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 13 * 13, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 25)
        )

    def forward(self, x):
        x = self.features_extractor(x)
        x = self.classifier(x)
        return x

# Function to save checkpoints
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)

# Training function with checkpoint saving
def train(model, train_loader, val_loader, epochs, device, checkpoint_path='checkpoint.pth.tar'):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    start_epoch = 0

    # Load checkpoint if exists
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Loaded checkpoint from {checkpoint_path}, resuming training from epoch {start_epoch}")

    for epoch in range(start_epoch, epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device ,non_blocking=True)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')
        
        # Validate the model
        model.eval()
        corrects = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                total += labels.size(0)
                corrects += (preds == labels).sum().item()
        
        val_accuracy = corrects / total
        print(f'Validation Accuracy: {val_accuracy:.4f}')

        # Save checkpoint
        save_checkpoint({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
        }, checkpoint_path)

    print('Training complete')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 10

model = CustomCNN().to(device)
train(model, train_dataloader, val_dataloader, epochs, device)


 15%|█▌        | 108/704 [08:22<46:13,  4.65s/it]


KeyboardInterrupt: 

In [4]:
from PIL import Image
import os
directory = r'C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train'


    # List to store folder paths
folder_paths = []

for root, dirs, files in os.walk(directory):
    for dir_name in dirs:
            # Construct the full path of the folder
        folder_path = os.path.join(root, dir_name)
        folder_paths.append(folder_path)



# Resize images
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        img_path = os.path.join(folder_path, filename)
        with Image.open(img_path) as img:
            # Resize image
            img_resized = img.resize((416, 416))
            # Save resized image, overwriting the original file
            img_resized.save(img_path)
            print('1')

print("Images have been resized and replaced.")


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [9]:

# Path to your main directory



C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Asian-Green-Bee-Eater
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Brown-Headed-Barbet
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Cattle-Egret
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Common-Kingfisher
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Common-Myna
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Common-Rosefinch
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Common-Tailorbird
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Coppersmith-Barbet
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Forest-Wagtail
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Gray-Wagtail
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Hoopoe
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\House-Crow
C:\Users\sshak\Downloads\Seen Datasets1\Seen Datasets\train\Indian-Grey-Hornbill
C:\Users\sshak

['C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Asian-Green-Bee-Eater',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Brown-Headed-Barbet',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Cattle-Egret',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Common-Kingfisher',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Common-Myna',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Common-Rosefinch',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Common-Tailorbird',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Coppersmith-Barbet',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Forest-Wagtail',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Gray-Wagtail',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets\\train\\Hoopoe',
 'C:\\Users\\sshak\\Downloads\\Seen Datasets1

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os
import shutil

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),  
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()  # Convert image to tensor
    # Note: Normalize is not included here as it would alter the pixel values
])

# Create the dataset
dataset_path = r'C:\Users\sshak\Downloads\Seen Datasets\Seen Datasets'
train_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'val'), transform=transform)

# Create the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)

# Function to save images
def save_augmented_images(dataloader, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    for i, (images, labels) in enumerate(dataloader):
        for j, (image, label) in enumerate(zip(images, labels)):
            img = transforms.ToPILImage()(image)  # Convert tensor back to PIL Image
            class_dir = os.path.join(save_dir, str(label.item()))
            if not os.path.exists(class_dir):
                os.makedirs(class_dir)
            img.save(os.path.join(class_dir, f'augmented_{i}_{j}.jpg'))  # Save as JPG

# Save the augmented training images
train_save_dir = os.path.join(dataset_path, 'train_augmented')
save_augmented_images(train_dataloader, train_save_dir)

# Save the augmented validation images
val_save_dir = os.path.join(dataset_path, 'val_augmented')
save_augmented_images(val_dataloader, val_save_dir)

# Replace old dataset directories with new ones
# Replace training set
shutil.rmtree(os.path.join(dataset_path, 'train'))
shutil.move(train_save_dir, os.path.join(dataset_path, 'train'))

# Replace validation set
shutil.rmtree(os.path.join(dataset_path, 'val'))
shutil.move(val_save_dir, os.path.join(dataset_path, 'val'))

print("Dataset replacement completed.")


Dataset replacement completed.
